# Setup Build / Pull Request Server

## Notebook Imports

In [ ]:
from aws_request import *
from aws_util import *
import aws_util
from aws_volumes import *

## Check Spot Instance Request

First, if there are hosts that you want to configure that might not be on AWS (for example, they are in Liferay's internal cloud), configure those here.

In [ ]:
app_host_names = None

If nothing is specified, this will use the instances for the application generated by the previous notebook.

In [ ]:
if app_host_names is None:
    app_request = InstanceRequest('app')
    app_instances = app_request.get_fulfilled()

    app_host_names = [instance['PublicDnsName'] for instance in app_instances]

app_host_names

## Specify SSH User

You'll need to specify two users: one which will run regular commands and one which will run installation commands.

In [ ]:
#user_name = 'ubuntu'
#sudo_user_name = 'ubuntu'
#user_name = 'ec2-user'
#sudo_user_name = 'ec2-user'
user_name = 'tomcat'
sudo_user_name = 'root'

If they're not the same, let the user with `sudo` capabilities know about the regular user.

In [ ]:
aws_util.private_key_location = os.path.expanduser('~/.ssh/id_rsa')

In [ ]:
if user_name != sudo_user_name:
    with open('awscli/regular_user.txt', 'w') as script_user_file:
        script_user_file.write(user_name)

    upload_file(sudo_user_name, app_host_names, 'awscli/regular_user.txt')

## Configure AWS CLI

In [ ]:
if user_name != sudo_user_name:
    install_awscli(sudo_user_name, app_host_names)

<h3 style="color:crimson">Liferay accounts usually do not have access to make use of IAM instance profiles, so you will need to manually run `aws configure` in order for AWS CLI to have the proper credentials to access AWS resources (such as S3 buckets). Please do that before proceeding.</h3>

## Specify S3 Bucket

Specify a bucket containing all of our crontab scripts.

In [ ]:
bucket_name = 'mdang.lesa'

Here, we'll make sure that our bucket is registered to our workstation server.

In [ ]:
set_bucket(user_name, app_host_names, bucket_name)
set_bucket(sudo_user_name, app_host_names, bucket_name)

## Upload Configuration Files

In [ ]:
%%writefile scripts/get_config.sh
#!/bin/bash
S3_BUCKET=$(cat $HOME/bucket.txt)

# Install git

if [ "" != "$(uname -a | grep Ubuntu)" ]; then
    sudo apt-get install -y git
else
    sudo yum install -y git
fi

# Clone the liferay-faster-deploy repository

git clone https://github.com/holatuwol/liferay-faster-deploy.git redeploy
ln -s redeploy/crontab

In [ ]:
run_script(user_name, app_host_names, 'get_config.sh')

## Setup Build Tools

In [ ]:
%%writefile scripts/setup_java.sh
#!/bin/bash
S3_BUCKET=$(cat bucket.txt)

touch portal-ext.properties

if [ -f regular_user.txt ]; then
    REGULAR_USER=$(cat regular_user.txt)
else
    REGULAR_USER=$USER
fi

# Install Java 6, 7, and 8

if [ "" != "$(uname -a | grep Ubuntu)" ]; then
    sudo add-apt-repository ppa:webupd8team/java
    sudo apt-get update

    echo oracle-java6-installer shared/accepted-oracle-license-v1-1 select true | sudo /usr/bin/debconf-set-selections
    echo oracle-java7-installer shared/accepted-oracle-license-v1-1 select true | sudo /usr/bin/debconf-set-selections
    echo oracle-java8-installer shared/accepted-oracle-license-v1-1 select true | sudo /usr/bin/debconf-set-selections
    sudo apt-get install -y oracle-java6-installer oracle-java7-installer oracle-java8-installer
    sudo apt-get install -y oracle-java8-set-default
else
    mkdir java
    aws s3 sync s3://$S3_BUCKET/jdk/ java/

    cd java/

    for file in $(ls -1 | grep -F '.rpm' | sort -u); do
        echo $file
        sudo rpm -ivh --force $file
    done

    cd ..
fi

# Install Ant

ANT_VERSION=1.9.9

wget -qq http://archive.apache.org/dist/ant/binaries/apache-ant-${ANT_VERSION}-bin.tar.gz
tar -zxf apache-ant-${ANT_VERSION}-bin.tar.gz
rm -f apache-ant-${ANT_VERSION}-bin.tar.gz

mv apache-ant-${ANT_VERSION} ant
sudo mv ant /usr/share

sudo ln -s /usr/share/ant/bin/ant /usr/bin/

# Install build tools (ant, jq, nodejs)

if [ "" != "$(uname -a | grep Ubuntu)" ]; then
    sudo apt-get install -y jq dos2unix
    curl -sL https://deb.nodesource.com/setup_6.x | sudo -E bash -
    sudo apt-get install -y nodejs
else
    sudo yum install -y epel-release
    sudo yum install -y jq dos2unix unix2dos
    curl -sL https://rpm.nodesource.com/setup_6.x | sudo -E bash -
    sudo yum install -y nodejs
fi

In [ ]:
%time run_script(sudo_user_name, app_host_names, 'setup_java.sh')

## Specify Server Type

First, specify the user that the scripts should assume when running Git commands.

In [ ]:
github_users = [None]

Make sure we have the same number of users as servers.

In [ ]:
assert len(github_users) == len(app_host_names)

Copy the OAuth token configured for local commands to the remote server.

In [ ]:
oauth_token = !git config github.oauth-token

for github_user, app_host_name in zip(github_users, app_host_names):
    if github_user is None:
        continue

    with open('scripts/setup_remote.sh', 'w') as oauth_file:
        oauth_file.write('#!/bin/bash\n')
        oauth_file.write('git config --global github.pull-user %s\n' % github_user)
        oauth_file.write('git config --global github.oauth-token %s\n' % oauth_token[0])

    run_script(user_name, [app_host_name], 'setup_remote.sh')

## Setup Apache

Builds will be made available from a web server so they can be downloaded to the local machine in the event that you need to modify portal properties or update a database.

In [ ]:
%%writefile scripts/setup_apache.sh
#!/bin/bash

if [ -f regular_user.txt ]; then
    REGULAR_USER=$(cat regular_user.txt)
else
    REGULAR_USER=$USER
fi

PULL_USER=$(sudo su - $REGULAR_USER -c 'git config github.pull-user')

if [ "" != "$(uname -a | grep Ubuntu)" ]; then
    sudo apt-get install -y apache2
else
    sudo yum -y install httpd
fi

# Create index file for serving build archives

sudo ln -s /home/$REGULAR_USER/crontab/index.js /var/www/html/index.js
sudo ln -s /home/$REGULAR_USER/crontab/index.css /var/www/html/index.css

sudo rm /var/www/html/index.html
sudo ln -s /home/$REGULAR_USER/crontab/index.html /var/www/html/index.html

if [ "" != "$(uname -a | grep Ubuntu)" ]; then
    sudo cp /home/$REGULAR_USER/crontab/liferay.conf /etc/apache2/sites-available
    sudo a2ensite liferay
else
    sudo cp /home/$REGULAR_USER/crontab/liferay.conf /etc/httpd/conf.d
fi

cd /home/$REGULAR_USER/crontab
./sethosts
cd -

if [ "" != "$(uname -a | grep Ubuntu)" ]; then
    sudo service apache2 restart
else
    sudo service httpd restart
fi

# Make sure that the user's home folder is world readable

FOLDER=$HOME

while [ "/" != "$FOLDER" ]; do
    sudo chmod a+rx $FOLDER
    FOLDER=$(dirname $FOLDER)
done

# Create the builds folder

sudo mkdir /var/www/html/builds
sudo chown $REGULAR_USER:$REGULAR_USER /var/www/html/builds

# Move the archived builds to /mnt if available

if [ -d /mnt/lost+found ]; then
    sudo mv /var/www/html /mnt/
    sudo mkdir /var/www/html
    sudo mount --bind /mnt/html /var/www/html
fi

In [ ]:
%time run_script(sudo_user_name, app_host_names, 'setup_apache.sh')

## Remount Home

In general, it's more performant to do work on the local instance volumes rather than on the EBS block store. Therefore, move the home folder to the local instance volume.

In [ ]:
%%writefile scripts/remount_home.sh
#!/bin/bash

MOUNT=

if [ -d /xvdc/lost+found ]; then
    MOUNT=/xvdc
elif [ -d /mnt/lost+found ]; then
    MOUNT=/mnt
fi

if [ -f regular_user.txt ]; then
    REGULAR_USER=$(cat regular_user.txt)
else
    REGULAR_USER=$USER
fi

if [ "" != "$MOUNT" ]; then
    cd /

    sudo mv /home/$REGULAR_USER $MOUNT/
    sudo mkdir /home/$REGULAR_USER
    sudo chown $REGULAR_USER:$REGULAR_USER /home/$REGULAR_USER
    sudo mount --bind $MOUNT/$REGULAR_USER /home/$REGULAR_USER
fi

In [ ]:
%time run_script(sudo_user_name, app_host_names, 'remount_home.sh')

## Setup GitHub

In [ ]:
%%writefile scripts/setup_github.sh
#!/bin/bash
S3_BUCKET=$(cat bucket.txt)

# Create folder used for builds

mkdir builds temp
mkdir master 70x 62x 61x

# Download portal source snapshot

crontab/gitrestore

# Install Miniconda

wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod u+x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b

# Enable Bash aliases

if [ -f .profile ]; then
    echo -e '\nsource $HOME/crontab/aliases' >> .profile
else
    echo -e '\nsource $HOME/crontab/aliases' >> .bashrc
fi

In [ ]:
%time run_script(user_name, app_host_names, 'setup_github.sh')

## Setup Crontab

In [ ]:
%%writefile scripts/setup_crontab.sh
#!/bin/bash

PULL_USER=$(git config github.pull-user)

# Enable crontab

crontab -l > .crontab

REFERENCE_TZ=-8
SERVER_TZ=$(expr $(date +%z | tr -d '+') '/' 100)
RELATIVE_TZ=$(expr $REFERENCE_TZ '-' $SERVER_TZ)

if [ "" != "$PULL_USER" ]; then
    echo "0 0 * * * $HOME/crontab/cronbuild clean > $HOME/daily.txt" >> .crontab

    HOURLY_START_TIME=$(expr '(' 5 '-' $RELATIVE_TZ ')' '%' 24)
    HOURLY_END_TIME=$(expr '(' $HOURLY_START_TIME '+' 13 ')' '%' 24)

    if [[ 11 -gt $HOURLY_START_TIME ]]; then
        HOURLY_TIME="${HOURLY_START_TIME}-${HOURLY_END_TIME}"
    elif [[ 11 -eq $HOURLY_START_TIME ]]; then
        HOURLY_TIME="0,${HOURLY_START_TIME}-23"
    else
        HOURLY_TIME="0-${HOURLY_END_TIME},${HOURLY_START_TIME}-23"
    fi

    echo "0 $HOURLY_TIME * * * $HOME/crontab/cronbuild > $HOME/hourly.txt" >> .crontab
else
    echo "0 0 * * 0 $HOME/crontab/cronbuild clean > $HOME/weekly.txt" >> .crontab
fi

crontab .crontab

In [ ]:
%time run_script(user_name, app_host_names, 'setup_crontab.sh')

## Cleanup Scripts

In [ ]:
%%writefile scripts/cleanup_user.sh
#!/bin/bash

rm get_config.sh
rm install_awscli.sh
rm set_bucket.sh
rm setup_crontab.sh
rm setup_github.sh
rm setup_remote.sh

rm -- "$0"

In [ ]:
run_script(user_name, app_host_names, 'cleanup_user.sh')

In [ ]:
%%writefile scripts/cleanup_sudo.sh
#!/bin/bash

rm extra_storage.sh
rm install_awscli.sh
rm set_bucket.sh
rm setup_apache.sh
rm setup_java.sh

if [ -f regular_user.txt ]; then
    rm bucket.txt
    rm regular_user.txt
    rm portal-ext.properties
fi

rm get-pip.py
rm region.txt

rm -- "$0"

In [ ]:
run_script(sudo_user_name, app_host_names, 'cleanup_sudo.sh')